Загрузка данных

In [ ]:
!rm -Rf my_data
!wget https://github.com/al-filippov/dpo_iis/raw/main/data.tar.gz -O data.tar.gz
!mkdir -p my_data
!tar -xf data.tar.gz -C my_data
!rm data.tar.gz

Установка зависимостей для извлечения текста из doc-файлов

In [ ]:
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils \
     pstotext tesseract-ocr \
     flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig libasound2-dev libpulse-dev
!pip install textract

Загрузка текста из документов

In [ ]:
import os
import textract
import itertools

def myprint(docs):
  count = 100
  doc = list(docs.keys())[0]
  example_value = list(docs.values())[0]
  value = None
  if isinstance(example_value, str):
    example_value = example_value.splitlines(True)
    output_count = min(count, len(example_value))
    value = '\n'.join(example_value[:output_count])
  elif isinstance(example_value, list):
    output_count = min(count, len(example_value))
    value = example_value[:output_count]
  elif isinstance(example_value, dict):
    output_count = min(count, len(example_value))
    value = list(itertools.islice(example_value.items(), output_count))
  print(f'Doc: {doc}\nContent: {value}')

docs = {}
dir_path = "my_data"

for path in os.listdir(dir_path):
    file_path = os.path.join(dir_path, path)
    if os.path.isfile(os.path.join(dir_path, path)):
       docs[path] = textract.process(file_path, encoding = 'unicode_escape').decode("utf8")
myprint(docs)

Установка библиотеки и модели для обработки текста

In [ ]:
!pip install spacy==3.5.2
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.5.0/ru_core_news_sm-3.5.0.tar.gz

Предобработка текста -- токенизация

In [32]:
import ru_core_news_sm

model = ru_core_news_sm.load()

t_docs = {}

for doc, text in docs.items():
  tokens = model(text)
  t_docs[doc] = []
  for token in tokens:
    str_token = str(token)
    if (len(str_token.strip()) > 0):
      t_docs[doc].append(token)

myprint(t_docs)

Doc: Этапы разработки проекта2.doc
Content: [Этапы, разработки, проекта, :, заключительные, стадии, проектирования, ,, схема, базы, данных, 1, Заключительные, стадии, проектирования, 1, Проектирование, процесса, тестирования, Проектирование, процесса, тестирования, ,, как, правило, ,, следует, за, процессом, функционального, проектирования, и, проектирования, схемы, базы, данных, ., На, этом, этапе, можно, использовать, сложные, схемы, тестирования, ,, а, можно, ограничиться, и, простыми, ., Здесь, мы, приведем, некоторые, принципы, ,, которых, нужно, придерживаться, при, проектировании, любой, информационной, системы, ., Когда, генерация, модуля, завершена, ,, выполняют, автономный, тест, ,, который, преследует, две, основные, цели, :, •, обнаружение, отказов, модуля, (, жестких, сбоев, ), ;, •, соответствие, модуля, спецификации, (, наличие, всех, необходимых, функций]


Предобработка текста -- удаление спец. символов

In [33]:
import re

regex = re.compile('[^a-zA-Zа-яА-Я]')

sp_docs = {}

for doc, tokens in t_docs.items():
  sp_docs[doc] = []
  for token in tokens:
    if len(regex.sub('', str(token))) > 2:
      sp_docs[doc].append(token)

myprint(sp_docs)

Doc: Этапы разработки проекта2.doc
Content: [Этапы, разработки, проекта, заключительные, стадии, проектирования, схема, базы, данных, Заключительные, стадии, проектирования, Проектирование, процесса, тестирования, Проектирование, процесса, тестирования, как, правило, следует, процессом, функционального, проектирования, проектирования, схемы, базы, данных, этом, этапе, можно, использовать, сложные, схемы, тестирования, можно, ограничиться, простыми, Здесь, приведем, некоторые, принципы, которых, нужно, придерживаться, при, проектировании, любой, информационной, системы, Когда, генерация, модуля, завершена, выполняют, автономный, тест, который, преследует, две, основные, цели, обнаружение, отказов, модуля, жестких, сбоев, соответствие, модуля, спецификации, наличие, всех, необходимых, функций, отсутствие, лишних, функций, После, того, как, автономный, тест, прошел, успешно, группа, сгенерированных, модулей, проходит, тесты, связей, которые, должны, отследить, взаимное, влияние, модулей, 

Предобработка текста -- удаление стоп-слов

In [34]:
stopwords = model.Defaults.stop_words
print(stopwords)

{'этакий', 'свыше', 'никакой', 'всячески', 'казалось', 'им', 'которые', 'настоящая', 'наперекор', 'стал', 'еле', 'эту', 'если', 'мог', 'мало', 'но', 'моих', 'там', 'у', 'бывают', 'твоя', 'вы', 'ша', 'до', 'сейчас', 'иль', 'позже', 'недавно', 'вернее', 'других', 'хотеть', 'щас', 'своими', 'ешь', 'мной', 'почти', 'довольно', 'сей', 'которым', 'скорее', 'имело', 'должны', 'для', 'сначала', 'ага', 'ишь', 'ниоткуда', 'одних', 'имъ', 'казаться', 'либо', 'накануне', 'потом', 'мы', 'точно', 'вновь', 'навряд', 'благодаря', 'ту', 'саму', 'бывала', 'том', 'недалеко', 'него', 'необходимо', 'своего', 'эка', 'ых', 'кроме', 'всегда', 'собой', 'ещё', 'напротив', 'ныне', 'дополнительно', 'взаимно', 'имела', 'которой', 'был', 'под', 'нашей', 'поэтому', 'одна', 'пусть', 'раньше', 'своя', 'вплоть', 'такие', 'явно', 'которая', 'нее', 'любая', 'своё', 'неё', 'твоё', 'самих', 'которому', 'л', 'далее', 'чего', 'бытует', 'вами', 'вовсе', 'общую', 'ой', 'ела', 'таки', 'неважно', 'явных', 'этою', 'напрямую', 'пе

In [35]:
st_docs = {}

for doc, tokens in sp_docs.items():
  st_docs[doc] = []
  for token in tokens:
    if not token.is_stop:
      st_docs[doc].append(token)

myprint(st_docs)

Doc: Этапы разработки проекта2.doc
Content: [Этапы, разработки, проекта, заключительные, стадии, проектирования, схема, базы, Заключительные, стадии, проектирования, Проектирование, процесса, тестирования, Проектирование, процесса, тестирования, правило, процессом, функционального, проектирования, проектирования, схемы, базы, этапе, использовать, сложные, схемы, тестирования, ограничиться, простыми, приведем, принципы, придерживаться, проектировании, информационной, системы, генерация, модуля, завершена, выполняют, автономный, тест, преследует, две, основные, цели, обнаружение, отказов, модуля, жестких, сбоев, соответствие, модуля, спецификации, наличие, необходимых, функций, отсутствие, лишних, функций, автономный, тест, прошел, успешно, группа, сгенерированных, модулей, проходит, тесты, связей, отследить, взаимное, влияние, модулей, группа, модулей, тестируется, надежность, работы, проходят, первых, тесты, имитации, отказов, системы, вторых, тесты, наработки, отказ, Первая, группа, т

Предобработка текста -- лемматизация

In [42]:
l_docs = {}

for doc, tokens in st_docs.items():
  l_docs[doc] = []
  for token in tokens:
    l_docs[doc].append(regex.sub('', str(token.lemma_).lower()))

myprint(l_docs)

Doc: Этапы разработки проекта2.doc
Content: ['этап', 'разработка', 'проект', 'заключительный', 'стадия', 'проектирование', 'схема', 'база', 'заключительный', 'стадия', 'проектирование', 'проектирование', 'процесс', 'тестирование', 'проектирование', 'процесс', 'тестирование', 'правило', 'процесс', 'функциональный', 'проектирование', 'проектирование', 'схема', 'база', 'этап', 'использовать', 'сложный', 'схема', 'тестирование', 'ограничиться', 'простой', 'привести', 'принцип', 'придерживаться', 'проектирование', 'информационный', 'система', 'генерация', 'модуль', 'завершить', 'выполнять', 'автономный', 'тест', 'преследовать', 'два', 'основный', 'цель', 'обнаружение', 'отказ', 'модуль', 'жсткий', 'сбой', 'соответствие', 'модуль', 'спецификация', 'наличие', 'необходимый', 'функция', 'отсутствие', 'лишний', 'функция', 'автономный', 'тест', 'пройти', 'успешно', 'группа', 'сгенерированных', 'модуль', 'проходить', 'тест', 'связь', 'отследить', 'взаимный', 'влияние', 'модуль', 'группа', 'модуль'

Индексирование документов -- создание словаря

In [43]:
unique_words = set()

for tokens in l_docs.values():
  unique_words |= set(tokens)
  

print(unique_words)

{'sqlcode', 'сбой', 'тайна', 'социология', 'запланировать', 'бланк', 'уровни', 'named', 'пристгиваться', 'здание', 'популярнейшей', 'пытаться', 'правил', 'sna', 'patentindex', 'indexstats', 'являться', 'узкий', 'стрелка', 'забытый', 'сравнение', 'приведение', 'следовательно', 'programming', 'rename', 'интегральность', 'пониматься', 'сомнительный', 'current', 'витрина', 'снаrsize', 'hour', 'усилие', 'покупаемого', 'придумывать', 'списывать', 'язвительный', 'columns', 'сентябрь', 'хотеть', 'рукав', 'решенными', 'partindex', 'пара', 'явный', 'ускорять', 'сервисный', 'огранизация', 'account', 'глас', 'описанными', 'увлечение', 'вложение', 'набрать', 'правила', 'ряд', 'indexes', 'помеха', 'выполняющего', 'обращение', 'требуемые', 'тезиса', 'утратить', 'стол', 'debugon', 'revoke', 'легко', 'триггер', 'экспуатировать', 'прочитанные', 'обеспечивающего', 'баланс', 'сотрудник', 'возможность', 'анализируетcя', 'рассматриваемую', 'турагентов', 'пригодный', 'извне', 'сжаты', 'многоуровневый', 'незн

Индексирование документов -- векторизация

In [44]:
ind_docs = {}

for doc in l_docs.keys():
  ind_docs[doc] = dict.fromkeys(unique_words, 0)

myprint(ind_docs)

for doc, tokens in l_docs.items():
  for token in tokens:
    ind_docs[doc][token] += 1

myprint(ind_docs)

Doc: Этапы разработки проекта2.doc
Content: [('sqlcode', 0), ('сбой', 0), ('тайна', 0), ('социология', 0), ('запланировать', 0), ('бланк', 0), ('уровни', 0), ('named', 0), ('пристгиваться', 0), ('здание', 0), ('популярнейшей', 0), ('пытаться', 0), ('правил', 0), ('sna', 0), ('patentindex', 0), ('indexstats', 0), ('являться', 0), ('узкий', 0), ('стрелка', 0), ('забытый', 0), ('сравнение', 0), ('приведение', 0), ('следовательно', 0), ('programming', 0), ('rename', 0), ('интегральность', 0), ('пониматься', 0), ('сомнительный', 0), ('current', 0), ('витрина', 0), ('снаrsize', 0), ('hour', 0), ('усилие', 0), ('покупаемого', 0), ('придумывать', 0), ('списывать', 0), ('язвительный', 0), ('columns', 0), ('сентябрь', 0), ('хотеть', 0), ('рукав', 0), ('решенными', 0), ('partindex', 0), ('пара', 0), ('явный', 0), ('ускорять', 0), ('сервисный', 0), ('огранизация', 0), ('account', 0), ('глас', 0), ('описанными', 0), ('увлечение', 0), ('вложение', 0), ('набрать', 0), ('правила', 0), ('ряд', 0), ('in

Индексирование документов -- TF (частота термина/токена)

In [45]:
def compute_tf(tokens_dict, tokens):
    tf_dict = {}
    tokens_count = len(tokens)
    for word, count in tokens_dict.items():
        tf_dict[word] = count / float(tokens_count)
    return tf_dict

tf_docs = {}

for doc, tokens_dict in ind_docs.items():
  tf_docs[doc] = compute_tf(tokens_dict, l_docs[doc])

myprint(tf_docs)

Doc: Этапы разработки проекта2.doc
Content: [('sqlcode', 0.0), ('сбой', 0.002670940170940171), ('тайна', 0.0), ('социология', 0.0), ('запланировать', 0.0), ('бланк', 0.0), ('уровни', 0.0), ('named', 0.0), ('пристгиваться', 0.0), ('здание', 0.0), ('популярнейшей', 0.0), ('пытаться', 0.0), ('правил', 0.0), ('sna', 0.0), ('patentindex', 0.0), ('indexstats', 0.0), ('являться', 0.004807692307692308), ('узкий', 0.0), ('стрелка', 0.0), ('забытый', 0.0), ('сравнение', 0.0008012820512820513), ('приведение', 0.0), ('следовательно', 0.0), ('programming', 0.0), ('rename', 0.0), ('интегральность', 0.0), ('пониматься', 0.0), ('сомнительный', 0.0), ('current', 0.0), ('витрина', 0.0), ('снаrsize', 0.0), ('hour', 0.0), ('усилие', 0.0), ('покупаемого', 0.0), ('придумывать', 0.0), ('списывать', 0.0), ('язвительный', 0.0), ('columns', 0.0), ('сентябрь', 0.0), ('хотеть', 0.0), ('рукав', 0.0), ('решенными', 0.0002670940170940171), ('partindex', 0.0), ('пара', 0.0), ('явный', 0.0005341880341880342), ('ускоря

Индексирование документов -- IDF (обратная документная частота термина/токена)

In [46]:
import math
    
docs_count = len(tf_docs.keys())

idf_dict = dict.fromkeys(unique_words, 0)
for doc, tokens_dict in tf_docs.items():
    for token, freq in tokens_dict.items():
        if freq > 0:
            idf_dict[token] += 1

for token, freq in idf_dict.items():
    idf_dict[token] = math.log(docs_count / float(freq))

print(idf_dict)

{'sqlcode': 3.0204248861443626, 'сбой': 0.8803587226480917, 'тайна': 3.0204248861443626, 'социология': 3.713572066704308, 'запланировать': 2.614959778036198, 'бланк': 3.713572066704308, 'уровни': 3.713572066704308, 'named': 3.713572066704308, 'пристгиваться': 3.713572066704308, 'здание': 3.713572066704308, 'популярнейшей': 3.713572066704308, 'пытаться': 1.7676619176489945, 'правил': 3.0204248861443626, 'sna': 3.713572066704308, 'patentindex': 3.713572066704308, 'indexstats': 3.713572066704308, 'являться': 0.050010420574661416, 'узкий': 2.327277705584417, 'стрелка': 2.614959778036198, 'забытый': 3.0204248861443626, 'сравнение': 1.5163474893680884, 'приведение': 2.327277705584417, 'следовательно': 1.7676619176489945, 'programming': 2.614959778036198, 'rename': 3.0204248861443626, 'интегральность': 3.713572066704308, 'пониматься': 2.614959778036198, 'сомнительный': 3.713572066704308, 'current': 3.0204248861443626, 'витрина': 3.0204248861443626, 'снаrsize': 3.713572066704308, 'hour': 3.713

Индексирование документов -- TF*IDF




In [47]:
tfidf_docs = {}
for doc, tokens_dict in tf_docs.items():
  tfidf_docs[doc] = {}
  for token, freq in tokens_dict.items():
    tfidf_docs[doc][token] = freq * idf_dict[token]

myprint(tfidf_docs)

Doc: Этапы разработки проекта2.doc
Content: [('sqlcode', 0.0), ('сбой', 0.0023513854771583646), ('тайна', 0.0), ('социология', 0.0), ('запланировать', 0.0), ('бланк', 0.0), ('уровни', 0.0), ('named', 0.0), ('пристгиваться', 0.0), ('здание', 0.0), ('популярнейшей', 0.0), ('пытаться', 0.0), ('правил', 0.0), ('sna', 0.0), ('patentindex', 0.0), ('indexstats', 0.0), ('являться', 0.00024043471430125682), ('узкий', 0.0), ('стрелка', 0.0), ('забытый', 0.0), ('сравнение', 0.0012150220267372503), ('приведение', 0.0), ('следовательно', 0.0), ('programming', 0.0), ('rename', 0.0), ('интегральность', 0.0), ('пониматься', 0.0), ('сомнительный', 0.0), ('current', 0.0), ('витрина', 0.0), ('снаrsize', 0.0), ('hour', 0.0), ('усилие', 0.0), ('покупаемого', 0.0), ('придумывать', 0.0), ('списывать', 0.0), ('язвительный', 0.0), ('columns', 0.0), ('сентябрь', 0.0), ('хотеть', 0.0), ('рукав', 0.0), ('решенными', 0.000991872881064185), ('partindex', 0.0), ('пара', 0.0), ('явный', 0.0012432039025557786), ('уско

Кластеризация -- подготовка данных

In [ ]:
!pip install fuzzy-c-means

In [49]:
import numpy as np
from matplotlib import pyplot as plt

data = []

for tokens_dict in tfidf_docs.values():
  data.append(list(tokens_dict.values()))

data = np.array(data)
print(data)

[[0.         0.00235139 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00018808 0.         0.00112193]
 ...
 [0.         0.00335162 0.         ... 0.00079001 0.         0.        ]
 [0.         0.         0.         ... 0.00025982 0.         0.        ]
 [0.         0.         0.         ... 0.00163394 0.         0.        ]]


Кластеризация -- разбиение документов по группам с помощью алгоритма FCM (Fuzzy C-Means)

In [56]:
from fcmeans import FCM

clusters = 2
fcm_model = FCM(n_clusters = clusters, m = 1.3)
fcm_model.fit(data)

centers = fcm_model.centers
labels = fcm_model.predict(data)

for cl in range(0, clusters):
  items = []
  for index, label in enumerate(labels):  
    if label == cl:
      items.append(list(tfidf_docs.keys())[index])
  print(f'Cluster {cl + 1}:\n{sorted(items)}')

Cluster 1:
['Архитектура, управляемая модель.doc', 'Введение в проектирование ИС.doc', 'Встроенные операторы SQL.doc', 'Методологии разработки программного обеспечения 2.doc', 'Методологии разработки программного обеспечения.doc', 'Методы композиции и декомпозиции.doc', 'Модели представления данных в СУБД.doc', 'Некоторые особенности проектирования.doc', 'Процедурное расширение языка SQL.doc', 'Системные объекты базы данных.doc', 'Технология создания распр ИС.doc', 'Требования к проекту.doc', 'Условия целостности БД.doc', 'Характеристики СУБД.doc', 'Этапы разработки проекта1.doc', 'Этапы разработки проекта2.doc', 'Этапы разработки проекта3.doc', 'Этапы разработки проекта4.doc', 'Этапы разработки проекта5.doc', 'Язык манипуляции данными.doc']
Cluster 2:
['tz_01.doc', 'tz_02.doc', 'tz_03.doc', 'tz_04.doc', 'tz_05.doc', 'tz_06.doc', 'tz_07.doc', 'tz_08.doc', 'tz_09.doc', 'tz_10.doc', 'tz_11.doc', 'tz_12.doc', 'tz_13.doc', 'tz_14.doc', 'tz_15.doc', 'tz_16.doc', 'tz_17.doc', 'tz_18.doc', 't